In [ ]:
!pip install opencv-python-headless==4.5.4.60
!pip install easyocr==1.6.2
!pip install pdf2image
!apt-get install poppler-utils
!pip install PyMuPDF jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 25.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 41.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached 

In [ ]:
#if you need to convert the pdf file to txt file
!pip install pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.0 MB/s eta 0:00:00


In [ ]:
#Trying to controll that the extracted text looks the same way as in image

import easyocr
import numpy as np
from pdf2image import convert_from_path
import xml.etree.ElementTree as ET

pdf_path = "2.pdf"

def sort_and_group_texts(texts):
    # Sort texts based on the top-left y-coordinate, then by x-coordinate
    sorted_texts = sorted(texts, key=lambda x: (x[0][0][1], x[0][0][0]))

    # Group texts by rows based on y-coordinate
    rows = []
    current_row = []
    last_y = None
    y_threshold = 10  # Adjust as necessary for your documents

    for bbox, text, _ in sorted_texts:
        top_left = bbox[0]
        y = top_left[1]

        if last_y is not None and abs(y - last_y) > y_threshold:
            # New row
            rows.append(current_row)
            current_row = []

        current_row.append((bbox, text))
        last_y = y

    # Add the last row if it's not empty
    if current_row:
        rows.append(current_row)

    return rows

try:
    reader = easyocr.Reader(['sv'])
    images = convert_from_path(pdf_path)

    all_rows = []

    for i, image in enumerate(images):
        image_np = np.array(image)
        image_rgb = image_np[:, :, ::-1].copy()  # Convert BGR to RGB
        detected_texts = reader.readtext(image_rgb, detail=1)  # Ensure detail is True for bbox
        rows = sort_and_group_texts(detected_texts)
        all_rows.extend(rows)

except Exception as e:
    print("Error:", e)

root = ET.Element("root")
page = ET.SubElement(root, "page")

for row in all_rows:
    row_element = ET.SubElement(page, "row")
    for bbox, text in row:
        item = ET.SubElement(row_element, "item")
        item.text = text
        item.set('bbox', str(bbox))
tree = ET.ElementTree(root)
tree.write("output.xml")
print("The process has been done, see your directory!")


The process has been done, see your directory!


In [ ]:
#Calculate CER and WER for each PDF
import os
import easyocr
from pdf2image import convert_from_path
import numpy as np
import re
from jiwer import wer, cer

def clean_text(text):
    """
    Remove invalid XML characters from the text.
    """
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

directory_path = 'text/'

reader = easyocr.Reader(['en'])

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(directory_path, filename)
        ground_truth_path = os.path.join(directory_path, filename.replace('.pdf', '.txt'))

        # Convert PDF to images
        images = convert_from_path(pdf_path)

        # Initialize variable to hold all OCR output text
        all_text = []

        # Perform OCR on each page
        for image in images:
            # Convert the PIL Image to a NumPy array
            np_image = np.array(image)

            # Use EasyOCR to read text from the image
            results = reader.readtext(np_image, paragraph=True)  # Using paragraph mode to better format text
            text = ' '.join([result[1] for result in results])
            clean_text_data = clean_text(text)
            all_text.append(clean_text_data)

        # Concatenate all OCR output text into a single string
        ocr_output_text = '\n'.join(all_text)

        # Load ground truth text
        with open(ground_truth_path, 'r', encoding='utf-8') as gt_file:
            ground_truth_text = gt_file.read()

        # Calculate WER and CER
        calculated_wer = wer(ground_truth_text, ocr_output_text)
        calculated_cer = cer(ground_truth_text, ocr_output_text)

        print(f"Results for {filename}:")
        print(f"\nWER: {calculated_wer:.2f}")
        print(f"CER: {calculated_cer:.2f}\n")


Results for 2EDEPZ4VHTLPTWSZR6FAVUJ3B2ZVSIPS.pdf:

WER: 0.21
CER: 0.08



In [ ]:
#convert pdf to txt file
import os
from pdfminer.high_level import extract_text

def convert_pdf_to_txt(pdf_path, txt_path):
    """
    Convert a PDF file to a text file using pdfminer.six.
    """
    try:
        text = extract_text(pdf_path)
        with open(txt_path, 'w', encoding='utf-8') as txt_file:
            txt_file.write(text)
        print(f"Converted {pdf_path} to {txt_path}")
    except Exception as e:
        print(f"Error converting {pdf_path}: {str(e)}")

def convert_pdfs_in_directory(directory):
    """
    Convert all PDF files to text files.
    """
    for filename in os.listdir(directory):
        if filename.lower().endswith('.pdf'):
            pdf_path = os.path.join(directory, filename)
            txt_path = os.path.join(directory, filename.replace('.pdf', '.txt'))
            convert_pdf_to_txt(pdf_path, txt_path)

def main():
    directory_path = 'large/'
    convert_pdfs_in_directory(directory_path)

if __name__ == "__main__":
    main()


Converted large/101 pages.pdf to large/101 pages.txt
Converted large/255 pages.pdf to large/255 pages.txt


In [ ]:
import os
import easyocr
from pdf2image import convert_from_path
import numpy as np
import re
from jiwer import wer, cer

def clean_text(text):
    """
    Remove invalid XML characters from the text.
    """
    invalid_xml_chars = re.compile(u'[\u0000-\u0008\u000B-\u000C\u000E-\u001F]')
    return invalid_xml_chars.sub('', text)

directory_path = 'table/'

reader = easyocr.Reader(['en'])
for filename in os.listdir(directory_path):
    if filename.endswith('.pdf'):
        pdf_path = os.path.join(directory_path, filename)
        ground_truth_path = os.path.join(directory_path, filename.replace('.pdf', '.txt'))

        # Convert PDF to images
        images = convert_from_path(pdf_path)

        # Initialize variable to hold all OCR output text
        all_text = []

        # Perform OCR on each page
        for image in images:
            # Convert the PIL Image to a NumPy array
            np_image = np.array(image)
            results = reader.readtext(np_image, paragraph=True)
            text = ' '.join([result[1] for result in results])
            clean_text_data = clean_text(text)
            all_text.append(clean_text_data)

        ocr_output_text = '\n'.join(all_text)

        # Load ground truth text
        with open(ground_truth_path, 'r', encoding='utf-8') as gt_file:
            ground_truth_text = gt_file.read()

        # Calculate WER and CER
        calculated_wer = wer(ground_truth_text, ocr_output_text)
        calculated_cer = cer(ground_truth_text, ocr_output_text)

        print(f"Results for {filename}:")
        print(f"\nWER: {calculated_wer:.2f}")
        print(f"CER: {calculated_cer:.2f}\n")


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% CompleteResults for 101 pages.pdf:

WER: 0.69
CER: 0.37



KeyboardInterrupt: 

In [ ]:
!unzip table-filename.zip

In [ ]:
!zip -r text-filename.zip text

In [ ]:
#Metadata Extraction

import easyocr
import numpy as np
import cv2
import xml.etree.ElementTree as ET
from pdf2image import convert_from_path
import re

def preprocess_image(image):
    """Apply preprocessing steps to improve OCR accuracy."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    thresh_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((2,2), np.uint8)
    dilated_image = cv2.dilate(thresh_image, kernel, iterations=1)
    return dilated_image

def extract_text_to_xml(pdf_path, xml_path):
    """Extracts text from images and saves to an XML file."""
    reader = easyocr.Reader(['sv'], recog_network='latin_g2')
    images = convert_from_path(pdf_path)
    root = ET.Element("root")

    for image in images:
        image_np = np.array(image)
        preprocessed_image = preprocess_image(image_np)
        results = reader.readtext(preprocessed_image, detail=1, paragraph=False)

        for bbox, text, confidence in results:
            item = ET.SubElement(root, "item")
            item.set('bbox', str(bbox))
            item.set('confidence', str(confidence))
            item.text = text

    tree = ET.ElementTree(root)
    tree.write(xml_path, encoding="utf-8", xml_declaration=True)

def extract_specific_info_from_xml(xml_path, keywords, text_limits):
    """Extracts specific information based on predefined keywords from an XML file."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    items = list(root.findall('item'))
    pairs = {}
    current_key = None
    current_text = []
    current_limit = None

    normalized_keywords = {keyword.lower(): keyword for keyword in keywords}

    for item in items:
        text = item.text or ""
        normalized_text = text.lower().strip()

        if any(normalized_text.startswith(keyword) for keyword in normalized_keywords):
            if current_key:
                formatted_text = ' '.join(current_text).strip()
                if current_limit is not None:
                    formatted_text = formatted_text[:current_limit]
                pairs[current_key] = formatted_text
            current_key = normalized_keywords.get(normalized_text, text.strip())
            current_text = []
            current_limit = text_limits.get(current_key, None)
        elif current_key:
            current_text.append(text.strip())

    if current_key:
        formatted_text = ' '.join(current_text).strip()
        if current_limit is not None:
            formatted_text = formatted_text[:current_limit]
        pairs[current_key] = formatted_text

    return pairs

def main():
    pdf_path = "3.pdf"
    xml_path = "output.xml"
    keywords = ["Fastighetsägare", "Namn", "Postnr och ort", "Fastighetsbeteckning", "Anläggare av ledning"]
    text_limits = {"Anläggare av ledning": 41}  # Limit text for last keyword

    extract_text_to_xml(pdf_path, xml_path)  # Extract text and save to XML
    info = extract_specific_info_from_xml(xml_path, keywords, text_limits)  # Process and display information

    for key in keywords:
        value = info.get(key, "")  # Use get to handle missing keywords gracefully
        print(f"{key}: {value}")

if __name__ == "__main__":
    main()


Fastighetsägare: Norsjö kommun ` (anläggarc) Îcins Luwpenrs Hliellaiby Wnh
Namn: ?8wgîR6  ! Adress 44sa _ iaZS '? Postnr och on1 Fastighctsbetcckning: 2a2s:'4:2
Postnr och ort: 
Fastighetsbeteckning: 
Anläggare av ledning: Norsjö kommun Storgatan 67 935 81 NORSJÖ 


In [ ]:
#Calculate the time duration
import easyocr
import numpy as np
import cv2
import xml.etree.ElementTree as ET
from pdf2image import convert_from_path
import time
def preprocess_image(image):
    """Apply preprocessing steps to improve OCR accuracy."""
    gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    thresh_image = cv2.adaptiveThreshold(blurred_image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((2,2), np.uint8)
    dilated_image = cv2.dilate(thresh_image, kernel, iterations=1)
    return dilated_image

def extract_text_to_xml(pdf_path, xml_path):
    """Extracts text from images and saves to an XML file."""
    reader = easyocr.Reader(['sv'], recog_network='latin_g2')
    images = convert_from_path(pdf_path)
    root = ET.Element("root")

    for image in images:
        image_np = np.array(image)
        preprocessed_image = preprocess_image(image_np)
        results = reader.readtext(preprocessed_image, detail=1, paragraph=False)

        for bbox, text, confidence in results:
            item = ET.SubElement(root, "item")
            item.set('bbox', str(bbox))
            item.set('confidence', str(confidence))
            item.text = text

    tree = ET.ElementTree(root)
    tree.write(xml_path, encoding="utf-8", xml_declaration=True)

def extract_specific_info_from_xml(xml_path, keywords, text_limits):
    """Extracts specific information based on predefined keywords from an XML file."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    items = list(root.findall('item'))
    pairs = {}
    current_key = None
    current_text = []
    current_limit = None

    normalized_keywords = {keyword.lower(): keyword for keyword in keywords}

    for item in items:
        text = item.text or ""
        normalized_text = text.lower().strip()

        if any(normalized_text.startswith(keyword) for keyword in normalized_keywords):
            if current_key:
                formatted_text = ' '.join(current_text).strip()
                if current_limit is not None:
                    formatted_text = formatted_text[:current_limit]
                pairs[current_key] = formatted_text
            current_key = normalized_keywords.get(normalized_text, text.strip())
            current_text = []
            current_limit = text_limits.get(current_key, None)
        elif current_key:
            current_text.append(text.strip())

    if current_key:
        formatted_text = ' '.join(current_text).strip()
        if current_limit is not None:
            formatted_text = formatted_text[:current_limit]
        pairs[current_key] = formatted_text

    return pairs

def main():
    start_time = time.time()  # Start timing here

    pdf_path = "4.pdf"
    xml_path = "output.xml"
    keywords = ["Fastighetsägare", "Namn", "Postnr och ort", "Fastighetsbeteckning", "Anläggare av ledning"]
    text_limits = {"Anläggare av ledning": 41}  # Limit text for "Anläggare av ledning"

    extract_text_to_xml(pdf_path, xml_path)
    info = extract_specific_info_from_xml(xml_path, keywords, text_limits)  # Process and display information

    for key in keywords:
        value = info.get(key, "")  # Use get to handle missing keywords gracefully
        print(f"{key}: {value}")

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total processing time: {elapsed_time} seconds")

if __name__ == "__main__":
    main()


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete